**В этом ноутбуке проведем тест обученной модели на детекцию лиц (скачана из https://github.com/akanametov/yolov8-face)**

In [1]:

import os

import requests
import cv2
import matplotlib.pyplot as plt
import glob 
# import random

import pickle

import pandas as pd
import numpy as np

# подключение гугл диска
from google.colab import drive
drive.mount('/content/drive')

# скрытие предупреждений
import warnings
warnings.filterwarnings("ignore")

# тут загрузим библиотеки для распознавания лиц

!pip install face_recognition
import face_recognition

from tqdm import tqdm
from PIL import Image, ImageDraw
from google.colab.patches import cv2_imshow
import pathlib
from pathlib import Path

!pip install ultralytics
from ultralytics import YOLO

import itertools
import math
import warnings
!pip install mediapy
import mediapy as media

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, \
Dropout, BatchNormalization
from tensorflow.keras.applications.vgg19 import preprocess_input

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566185 sha256=b134f4f6d9145583527d6cf9a5579a8685d78abdba0e09ae3351b0b5023c8378
  Stored in directory: /root/.cache/pip/wheels/22/a8/60/4a2aeb763d63f50190f4c4e07069a22245347eeafdb3a67551
Successfully built face-recognition-models
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.3/488.3 KB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 KB 1.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1

Загрузим пользовательское видео

In [2]:
!cp /content/drive/MyDrive/diplom/video_2023-03-28_21-27-12-2.mp4 /content/

Загрузим веса модели

In [3]:
!cp /content/drive/MyDrive/diplom/yolov8n-face.pt /content/

Ранее обученная модель VGG19 на классификацию эмоций

In [4]:
path = Path("/content/drive/My Drive/diplom/model_save_ocv/checkpoint_best128_new_DS_24-02.h5")

Создадим класс для работы VGG19

In [5]:

class em_model:
    def __init__(self, config:str):
        self.config = config
        self.model = tf.keras.models.load_model(self.config)
        # warm-up model on a random sample
        self.img_size = 128
        sample = tf.random.uniform((1, self.img_size, self.img_size, 3), 0, 1)
        self.model(sample)
        self.class_names = {0: 'anger',
                      1: 'contempt',
                      2: 'disgust',
                      3: 'fear',
                      4: 'happy',
                      5: 'neutral',
                      6: 'sad',
                      7: 'surprise',
                      8: 'uncertain'}
        
    def predict_model(self, image):
        input_arr = cv2.resize(image, (128,128), interpolation=cv2.INTER_AREA)
        input_arr = preprocess_input(input_arr)
        input_arr = np.array([input_arr])  # Convert single image to a batch.
          
        predictions = self.model.predict(input_arr)

        emotion = self.class_names[np.argmax(predictions)]
        return emotion

Объявление моделей детекции и классификации

In [6]:
model_path = '/content/yolov8n-face.pt'

In [7]:
model_emotion = em_model(path)

model = YOLO(model_path)

WARNING ⚠️ /content/yolov8n-face.pt appears to require 'omegaconf', which is not in ultralytics requirements.
AutoInstall will run now for 'omegaconf' but this feature will be removed in the future.
Recommend fixes are to train a new model using the latest 'ultralytics' package or to run a command with an official YOLOv8 model, i.e. 'yolo predict model=yolov8n.pt'
requirements: YOLOv8 requirement "omegaconf" not found, attempting AutoUpdate...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 KB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144573 sha256=8aad1f8524ae4445ed2ed07e97977e804ca34caa663978e8a6c28dd36917b0d4
  Stor

In [8]:
video_path = '/content/video_2023-03-28_21-27-12-2.mp4'

Вспомогательная функция для парсинга видеофайла

In [9]:
from moviepy.editor import VideoFileClip
import numpy as np
import os
from datetime import timedelta

In [10]:
# i.e if video of duration 30 seconds, saves 10 frame per second = 300 frames saved in total
SAVING_FRAMES_PER_SECOND = 10

In [11]:
def format_timedelta(td):
    """Utility function to format timedelta objects in a cool way (e.g 00:00:20.05) 
    omitting microseconds and retaining milliseconds"""
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return result + ".00".replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":", "-")

Функция для отрисовки названия эмоций (чтобы закрашивался фон текста)

In [12]:
def rect(img,x,y,w,h,text=""):
    # cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
    if text != "":
      l = len(text)*35
      cv2.rectangle(img,(x,y-44),(x+l,y),(0,255,0),-1)
      font = cv2.FONT_HERSHEY_SIMPLEX
      cv2.putText(img,text,(x,y), font, 2,(0,0,0),2,cv2.LINE_AA)

Функция парсинга - раскладывает кадры сохраняет в отдельную папку и параллельно формирует новый видео файл с распознанными лицами и эмоциями

In [13]:
%%capture --no-display
def main(video_file):
    SAVING_FRAMES_PER_SECOND = 25
    # load the video clip
    video_clip = VideoFileClip(video_file)
    # make a folder by the name of the video file
    filename, _ = os.path.splitext(video_file)
    filename += "-moviepy"
    if not os.path.isdir(filename):
        os.mkdir(filename)

    height, width, layers = (1080, 1920, 3)
    frameSize = (width,height)
    out = cv2.VideoWriter('output_video1.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 25, frameSize) 
    
    
    # if the SAVING_FRAMES_PER_SECOND is above video FPS, then set it to FPS (as maximum)
    saving_frames_per_second = min(video_clip.fps, SAVING_FRAMES_PER_SECOND)
    # if SAVING_FRAMES_PER_SECOND is set to 0, step is 1/fps, else 1/SAVING_FRAMES_PER_SECOND
    step = 1 / video_clip.fps if saving_frames_per_second == 0 else 1 / saving_frames_per_second
    # iterate over each possible frame
    for current_duration in np.arange(0, video_clip.duration, step):
        # format the file name and save it
        frame_duration_formatted = format_timedelta(timedelta(seconds=current_duration)).replace(":", "-")
        frame_filename = os.path.join(filename, f"frame{frame_duration_formatted}.jpg")
        # save the frame with the current duration
        video_clip.save_frame(frame_filename, current_duration)

        img = cv2.imread(frame_filename)
        results = model.predict(source=frame_filename, show=False, save=False, conf=0.25, imgsz=640, line_thickness=1, max_det=1000) 
        box_array = 0

        # count = 0 
        img = results[0].orig_img
        bxes = results[0].boxes

        
        for x, y, w, h in bxes.xyxy.tolist():
          x, y, w, h = int(x), int(y), int(w), int(h)
          # print('bsagas12121',x, y, w, h, frame_count)
          img_box = img[y:h, x:w, :]

          label = model_emotion.predict_model(img_box);
          cv2.rectangle(img,(x, y),(w, h),(255,0,0),5)
          rect(img, x, y, w, h, text = label)
          


        out.write(img) 
        cv2.imwrite(frame_filename, img)

    out.release()






In [14]:
main(video_path)


image 1/1 /content/video_2023-03-28_21-27-12-2-moviepy/frame0-00-00.00.jpg: 384x640 2 faces, 64.5ms
Speed: 0.9ms preprocess, 64.5ms inference, 67.9ms postprocess per image at shape (1, 3, 640, 640)


1/1 [==============================] - 1s 761ms/step



image 1/1 /content/video_2023-03-28_21-27-12-2-moviepy/frame0-00-00.04.jpg: 384x640 2 faces, 17.1ms
Speed: 0.7ms preprocess, 17.1ms inference, 9.2ms postprocess per image at shape (1, 3, 640, 640)


1/1 [==============================] - 0s 31ms/step



image 1/1 /content/video_2023-03-28_21-27-12-2-moviepy/frame0-00-00.08.jpg: 384x640 2 faces, 17.2ms
Speed: 0.6ms preprocess, 17.2ms inference, 2.4ms postprocess per image at shape (1, 3, 640, 640)


...

1/1 [==============================] - 0s 33ms/step



image 1/1 /content/video_2023-03-28_21-27-12-2-moviepy/frame0-00-12.28.jpg: 384x640 2 faces, 11.5ms
Speed: 0.5ms preprocess, 11.5ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)


1/1 [==============================] - 0s 20ms/step



image 1/1 /content/video_2023-03-28_21-27-12-2-moviepy/frame0-00-12.32.jpg: 384x640 2 faces, 11.0ms
Speed: 0.5ms preprocess, 11.0ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)


1/1 [==============================] - 1s 839ms/step


Сжимаем видео для отображения в ноутбуке

In [15]:
from IPython.display import HTML
from base64 import b64encode
import os

# Input video path
save_path = "/content/output_video1.mp4"

# Compressed video path
compressed_path = "/content/result_compressed.mp4"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Output hidden; open in https://colab.research.google.com to view.